In [285]:
import numpy as np
import pandas as pd

In [286]:
train_data = pd.read_csv('../input/marketing-strategy-personalised-offer/train_data.csv')
train_data.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No


In [287]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12379 entries, 0 to 12378
Data columns (total 31 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   offer expiration                      12379 non-null  object
 1   income_range                          12379 non-null  object
 2   no_visited_Cold drinks                12181 non-null  object
 3   travelled_more_than_15mins_for_offer  12379 non-null  int64 
 4   Restaur_spend_less_than20             12258 non-null  object
 5   Marital Status                        12379 non-null  object
 6   restaurant type                       12379 non-null  object
 7   age                                   12379 non-null  object
 8   Prefer western over chinese           12379 non-null  int64 
 9   travelled_more_than_25mins_for_offer  12379 non-null  int64 
 10  travelled_more_than_5mins_for_offer   12379 non-null  int64 
 11  no_visited_bars             

In [288]:
train_data.isnull().sum()

offer expiration                            0
income_range                                0
no_visited_Cold drinks                    198
travelled_more_than_15mins_for_offer        0
Restaur_spend_less_than20                 121
Marital Status                              0
restaurant type                             0
age                                         0
Prefer western over chinese                 0
travelled_more_than_25mins_for_offer        0
travelled_more_than_5mins_for_offer         0
no_visited_bars                            93
gender                                      0
car                                     12268
restuarant_same_direction_house             0
Cooks regularly                             0
Customer type                               0
Qualification                               0
is foodie                                   0
no_Take-aways                             144
Job/Job Industry                            0
restuarant_opposite_direction_hous

In [289]:
train_data[train_data['car'].notnull()]['Offer Accepted'].value_counts(),train_data['Offer Accepted'].value_counts()

(Yes    61
 No     50
 Name: Offer Accepted, dtype: int64,
 Yes    6994
 No     5385
 Name: Offer Accepted, dtype: int64)

In [290]:
train_data = train_data.drop(columns="car")

In [291]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="most_frequent")

In [293]:
imputer.fit(train_data)
train_data = pd.DataFrame(data=imputer.transform(train_data), columns=train_data.columns)

In [294]:
train_data.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No


In [295]:
feature_list = train_data.columns[:-1].values
target = [train_data.columns[-1]]
feature_list

array(['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer',
       'Restaur_spend_less_than20', 'Marital Status', 'restaurant type',
       'age', 'Prefer western over chinese',
       'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'restuarant_same_direction_house', 'Cooks regularly',
       'Customer type', 'Qualification', 'is foodie', 'no_Take-aways',
       'Job/Job Industry', 'restuarant_opposite_direction_house',
       'has Children', 'visit restaurant with rating (avg)',
       'temperature', 'Restaur_spend_greater_than20', 'Travel Time',
       'Climate', 'drop location', 'Prefer home food'], dtype=object)

In [296]:
binary_features = ['travelled_more_than_15mins_for_offer','Prefer western over chinese','travelled_more_than_25mins_for_offer','travelled_more_than_5mins_for_offer','gender','restuarant_same_direction_house','Cooks regularly','is foodie','restuarant_opposite_direction_house','has Children','Prefer home food']
categorical_features = ['Marital Status','restaurant type','Customer type','Qualification','Job/Job Industry','Climate','drop location']
ordinal_features = ['offer expiration','income_range','no_visited_Cold drinks','Restaur_spend_less_than20','age','no_visited_bars','no_Take-aways','visit restaurant with rating (avg)','temperature','Restaur_spend_greater_than20','Travel Time']

In [297]:
len(binary_features+categorical_features+ordinal_features) == len(feature_list)

True

In [298]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

In [299]:
ordinal_encoder.fit(train_data[categorical_features+ordinal_features])
train_data[categorical_features+ordinal_features] = ordinal_encoder.transform(train_data[categorical_features+ordinal_features])

In [300]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder(sparse=False)

In [301]:
#pd.get_dummies(train_data,columns=['gender'],drop_first=True)
cat_encoder.fit(train_data['gender'].unique().reshape(-1, 1))
#cat_encoder.transform(train_data['gender'].to_numpy().reshape(-1, 1))[:,0]
ohe_df = pd.DataFrame(cat_encoder.transform(train_data['gender'].to_numpy().reshape(-1, 1))[:,0], columns=['Gender'])
train_data['gender'] = ohe_df['Gender']

In [309]:
train_data[binary_features] = train_data[binary_features].astype('float64')

In [310]:
train_data[binary_features].head()

,travelled_more_than_15mins_for_offer,Prefer western over chinese,travelled_more_than_25mins_for_offer,travelled_more_than_5mins_for_offer,gender,restuarant_same_direction_house,Cooks regularly,is foodie,restuarant_opposite_direction_house,has Children,Prefer home food
0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
2,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [305]:
train_data[categorical_features].head()

,Marital Status,restaurant type,Customer type,Qualification,Job/Job Industry,Climate,drop location
0,1.0,1.0,0.0,1.0,24.0,0.0,1.0
1,1.0,4.0,0.0,1.0,24.0,1.0,1.0
2,2.0,2.0,0.0,1.0,1.0,2.0,0.0
3,2.0,4.0,0.0,5.0,21.0,1.0,2.0
4,2.0,1.0,0.0,5.0,21.0,1.0,2.0


In [306]:
train_data[ordinal_features].head()

,offer expiration,income_range,no_visited_Cold drinks,Restaur_spend_less_than20,age,no_visited_bars,no_Take-aways,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time
0,1.0,1.0,1.0,3.0,3.0,3.0,0.0,4.0,1.0,3.0,4.0
1,1.0,8.0,1.0,1.0,6.0,4.0,0.0,3.0,2.0,0.0,3.0
2,1.0,8.0,3.0,0.0,1.0,4.0,0.0,4.0,1.0,3.0,0.0
3,0.0,4.0,3.0,0.0,5.0,4.0,0.0,3.0,2.0,0.0,0.0
4,1.0,1.0,4.0,0.0,0.0,3.0,2.0,3.0,0.0,3.0,0.0


In [311]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12379 entries, 0 to 12378
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   offer expiration                      12379 non-null  float64
 1   income_range                          12379 non-null  float64
 2   no_visited_Cold drinks                12379 non-null  float64
 3   travelled_more_than_15mins_for_offer  12379 non-null  float64
 4   Restaur_spend_less_than20             12379 non-null  float64
 5   Marital Status                        12379 non-null  float64
 6   restaurant type                       12379 non-null  float64
 7   age                                   12379 non-null  float64
 8   Prefer western over chinese           12379 non-null  float64
 9   travelled_more_than_25mins_for_offer  12379 non-null  float64
 10  travelled_more_than_5mins_for_offer   12379 non-null  float64
 11  no_visited_bars

In [312]:
train_data.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,3.0,0.0,0.0,...,0.0,0.0,4.0,1.0,3.0,4.0,0.0,1.0,0.0,No
1,1.0,8.0,1.0,0.0,1.0,1.0,4.0,6.0,0.0,0.0,...,0.0,1.0,3.0,2.0,0.0,3.0,1.0,1.0,0.0,Yes
2,1.0,8.0,3.0,1.0,0.0,2.0,2.0,1.0,1.0,0.0,...,1.0,1.0,4.0,1.0,3.0,0.0,2.0,0.0,1.0,Yes
3,0.0,4.0,3.0,0.0,0.0,2.0,4.0,5.0,1.0,0.0,...,0.0,1.0,3.0,2.0,0.0,0.0,1.0,2.0,0.0,No
4,1.0,1.0,4.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,2.0,0.0,No
